# Train a Model to Trade on the Stock Market

![](https://image.ibb.co/ed08eS/Stock_Market_Investor.jpg)

## Efficient Market Hypothesis

Future prices cannot be predicted by analyzing prices from the past. Excess returns cannot be earned in the long run by using investment strategies based on historical share prices or other historical data. Technical analysis techniques will not be able to consistently produce excess returns.

 

## Technical Analysis


In finance, technical analysis is an analysis methodology for forecasting the direction of prices through the study of past market data, primarily price and volume. The efficacy of technical  analysis is disputed by the efficient-market hypothesis which states that stock market prices are essentially unpredictable.

 

## Can a neural net learn Technical Analysis techniques to predict the market and make good trades?

Are we about to get rich?

In [1]:
import random
from indicators import *
from transform import *
import pandas_datareader as web
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import h2o
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import cufflinks as cf
from plots import *
import numpy as np
import operator
import math

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Choose a Stock to Predict
Enter the stock symbol and the market it's traded on. For fun, enter a dollar value to theoretically invest. The window is a sliding time frame in days that the algorithm will learn from during training. When making predictions, it will help the model to provide data for this many days prior to your desired prediction, but it is not mandatory.

In [2]:
stockSymbol = "PINC"
market = "NASDAQ"
invest = 1000

window = 15
start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2017,1,1)
split_test = pd.datetime(2018,1,1)

### Connect to Morningstar and Download the Historical Data  

In [3]:
df = web.DataReader(stockSymbol, 'morningstar', start=start_date)
df = df.reset_index(drop=False)
df = df.drop('Symbol', axis=1)
df.set_index('Date', inplace=True)
df = df[['Open','High', 'Low', 'Close','Volume']]


### Plot.ly is a Sweet Charting Library
This chart shows historical price data for the chosen stock. Also, the vertical lines represet the time frames to be used for training, validation, and testing. 

In [4]:
myPlotlyKey = 'conh5EnFad0Z9Lz6mVWr'
py.sign_in(username='bretto777',api_key=myPlotlyKey)

fig = simpleStockChart(df,stockSymbol,split_validation,split_test)
py.iplot(fig, validate=False)

The draw time for this plot will be slow for clients without much RAM.


### Feature Engineering for Technical Analysis
In this step, we add "technical indicators" to enrich the data, providing the model with additional clues as to whether the stock might go up or down. These indicators are commonly used by pro investors to make better trading decisions. Hopefully our model will learn to do the same!

In [5]:
%%capture
df.reset_index(drop=False, inplace=True, col_level=0)

typical_price(df, high_col = 'High', low_col = 'Low', close_col = 'Close')
money_flow_index(df, vol_col='Volume')
rsi(df, close_col='Close')
ema(df, period=26, column='Close')
ema(df, period=12, column='Close')
macd(df, period_long=26, period_short=12, period_signal=9, column='Close')
on_balance_volume(df, trend_periods=21, close_col='Close', vol_col='Volume')
price_volume_trend(df, trend_periods=21, close_col='Close', vol_col='Volume')
bollinger_bands(df, trend_periods=20, close_col='Close')
williams_r(df, periods=14, high_col='High', low_col='Low', close_col='Close')
ultimate_oscillator(df, period_1=7,period_2=14, period_3=28, high_col='High', low_col='Low', close_col='Close')
df = df.drop(['uo_bp', 'uo_tr', 'uo_avg_1', 'uo_avg_2', 'uo_avg_3'], axis=1)

In [6]:
fig = technicalAnalysisChart(df)
py.iplot(fig, validate=False)

### Transform the Data
This sliding window function pivots our sequential data, such that time periods in the original data are represented as columns in the new dataframe. This transformation allows us to use a wide range of supervised machine learning algorithms such as GLMs, GBMs, Random Forests, and Neural Networks. 

![](https://image.ibb.co/hDU3Dn/sliding_Window.png)

In [7]:
windowTransform(df, window)
df.head(10)

,Date,Open,High,Low,Close,Volume,typical_price,money_flow,money_ratio,money_flow_index,money_flow_positive,money_flow_negative,rsi,ema26,ema12,macd_val,macd_signal_line,obv,obv_ema21,pvt,pvt_ema21,bol_bands_middle,bol_bands_upper,bol_bands_lower,williams_r,ultimate_oscillator,Close Tomorrow,Close Tomorrow Change,Close Tomorrow Trend,Open-1,Open Change,Open-1 Trend,Open-2,Open Change-1,Open-2 Trend,Open-3,Open Change-2,Open-3 Trend,Open-4,Open Change-3,Open-4 Trend,Open-5,Open Change-4,Open-5 Trend,Open-6,Open Change-5,Open-6 Trend,Open-7,Open Change-6,Open-7 Trend,Open-8,Open Change-7,Open-8 Trend,Open-9,Open Change-8,Open-9 Trend,Open-10,Open Change-9,Open-10 Trend,Open-11,Open Change-10,Open-11 Trend,Open-12,Open Change-11,Open-12 Trend,Open-13,Open Change-12,Open-13 Trend,Open-14,Open Change-13,Open-14 Trend,High-1,High Change,High-1 Trend,High-2,High Change-1,High-2 Trend,High-3,High Change-2,High-3 Trend,High-4,High Change-3,High-4 Trend,High-5,High Change-4,High-5 Trend,High-6,High Change-5,High-6 Trend,High-7,High Change-6,High-7 Trend,High-8,High Change-7,High-8 Trend,High-9,High Change-8,High-9 Trend,High-10,High Change-9,High-10 Trend,High-11,High Change-10,High-11 Trend,High-12,High Change-11,High-12 Trend,High-13,High Change-12,High-13 Trend,High-14,High Change-13,High-14 Trend,Low-1,Low Change,Low-1 Trend,Low-2,Low Change-1,Low-2 Trend,Low-3,Low Change-2,Low-3 Trend,Low-4,Low Change-3,Low-4 Trend,Low-5,Low Change-4,Low-5 Trend,Low-6,Low Change-5,Low-6 Trend,Low-7,Low Change-6,Low-7 Trend,Low-8,Low Change-7,Low-8 Trend,Low-9,Low Change-8,Low-9 Trend,Low-10,Low Change-9,Low-10 Trend,Low-11,Low Change-10,Low-11 Trend,Low-12,Low Change-11,Low-12 Trend,Low-13,Low Change-12,Low-13 Trend,Low-14,Low Change-13,Low-14 Trend,Close-1,Close Change,Close-1 Trend,Close-2,Close Change-1,Close-2 Trend,Close-3,Close Change-2,Close-3 Trend,Close-4,Close Change-3,Close-4 Trend,Close-5,Close Change-4,Close-5 Trend,Close-6,Close Change-5,Close-6 Trend,Close-7,Close Change-6,Close-7 Trend,Close-8,Close Change-7,Close-8 Trend,Close-9,Close Change-8,Close-9 Trend,Close-10,Close Change-9,Close-10 Trend,Close-11,Close Change-10,Close-11 Trend,Close-12,Close Change-11,Close-12 Trend,Close-13,Close Change-12,Close-13 Trend,Close-14,Close Change-13,Close-14 Trend,BBand-Upper-1,BBand-Upper Change,BBand-Upper-1 Trend,BBand-Lower-1,BBand-Lower Change,BBand-Lower-1 Trend,BBand-Middle-1,BBand-Middle Change,BBand-Middle-1 Trend,BBand-Upper-2,BBand-Upper Change-1,BBand-Upper-2 Trend,BBand-Lower-2,BBand-Lower Change-1,BBand-Lower-2 Trend,BBand-Middle-2,BBand-Middle Change-1,BBand-Middle-2 Trend,BBand-Upper-3,BBand-Upper Change-2,BBand-Upper-3 Trend,BBand-Lower-3,BBand-Lower Change-2,BBand-Lower-3 Trend,BBand-Middle-3,BBand-Middle Change-2,BBand-Middle-3 Trend,BBand-Upper-4,BBand-Upper Change-3,BBand-Upper-4 Trend,BBand-Lower-4,BBand-Lower Change-3,BBand-Lower-4 Trend,BBand-Middle-4,BBand-Middle Change-3,BBand-Middle-4 Trend,BBand-Upper-5,BBand-Upper Change-4,BBand-Upper-5 Trend,BBand-Lower-5,BBand-Lower Change-4,BBand-Lower-5 Trend,BBand-Middle-5,BBand-Middle Change-4,BBand-Middle-5 Trend,BBand-Upper-6,BBand-Upper Change-5,BBand-Upper-6 Trend,BBand-Lower-6,BBand-Lower Change-5,BBand-Lower-6 Trend,BBand-Middle-6,BBand-Middle Change-5,...,BBand-Middle-10 Trend,BBand-Upper-11,BBand-Upper Change-10,BBand-Upper-11 Trend,BBand-Lower-11,BBand-Lower Change-10,BBand-Lower-11 Trend,BBand-Middle-11,BBand-Middle Change-10,BBand-Middle-11 Trend,BBand-Upper-12,BBand-Upper Change-11,BBand-Upper-12 Trend,BBand-Lower-12,BBand-Lower Change-11,BBand-Lower-12 Trend,BBand-Middle-12,BBand-Middle Change-11,BBand-Middle-12 Trend,BBand-Upper-13,BBand-Upper Change-12,BBand-Upper-13 Trend,BBand-Lower-13,BBand-Lower Change-12,BBand-Lower-13 Trend,BBand-Middle-13,BBand-Middle Change-12,BBand-Middle-13 Trend,BBand-Upper-14,BBand-Upper Change-13,BBand-Upper-14 Trend,BBand-Lower-14,BBand-Lower Change-13,BBand-Lower-14 Trend,BBand-Middle-14,BBand-Middle Change-13,BBand-Middle-14 Tren

In [8]:
df['Close Tomorrow Trend'].replace(0,-1,inplace=True)    

trainingDF = df.loc[df['Date'] <= split_validation]
validationDF = df[df['Date'].isin(pd.date_range(split_validation,split_test))]
testDF = df.loc[df['Date'] > split_test]

trainingDF = trainingDF.dropna(axis=0, how='any')
validationDF = validationDF.dropna(axis=0, how='any')
testDF = testDF.fillna(value=0)

trainingDF.set_index('Date', inplace=True)
validationDF.set_index('Date', inplace=True)
testDF.set_index('Date', inplace=True)

### Let's use a machine learning library called H2O
H2O is an in-memory platform for distributed, scalable machine learning. H2O uses familiar interfaces like R, Python, Scala, Java, JSON and the Flow notebook/web interface, and works seamlessly with big data technologies like Hadoop and Spark. H2O provides implementations of many popular algorithms such as GBM, Random Forest, Deep Neural Networks, Word2Vec and Stacked Ensembles.

![](https://www.h2o.ai/wp-content/uploads/2017/11/h2o-architecture-1.png)


Start up the H2O engine

In [9]:
#h2o.cluster().shutdown()
h2o.init(max_mem_size="12G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_162"; OpenJDK Runtime Environment (build 1.8.0_162-8u162-b12-0ubuntu0.16.04.2-b12); OpenJDK 64-Bit Server VM (build 25.162-b12, mixed mode)
  Starting server from /home/brett/.local/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9NAqBM
  JVM stdout: /tmp/tmp9NAqBM/h2o_brett_started_from_python.out
  JVM stderr: /tmp/tmp9NAqBM/h2o_brett_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.7
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_brett_idzbka
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Get the data that we prepared earlier, and load it into H2O.

![](https://image.ibb.co/fo5VDn/data_To_H2_O.jpg)

In [10]:
train = h2o.H2OFrame(trainingDF, destination_frame="train.hex")
valid = h2o.H2OFrame(validationDF, destination_frame="valid.hex")
test = h2o.H2OFrame(testDF, destination_frame="test.hex")

train[train['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1   #Make sure there are no 0's. Enforce Bernoulli
train['Close Tomorrow Trend'] = train['Close Tomorrow Trend'].asfactor() #Make sure the target is categorical
train = train.drop('Close Tomorrow', axis=1)                             #Drop cheater columns
train = train.drop('Close Tomorrow Change', axis=1)                      #Drop

valid[valid['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
valid['Close Tomorrow Trend'] = valid['Close Tomorrow Trend'].asfactor()
valid = valid.drop('Close Tomorrow', axis=1)
valid = valid.drop('Close Tomorrow Change', axis=1)

test[test['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
test['Close Tomorrow Trend'] = test['Close Tomorrow Trend'].asfactor()
test = test.drop('Close Tomorrow', axis=1)
test = test.drop('Close Tomorrow Change', axis=1)

# Set predictor and response variables
y = 'Close Tomorrow Trend'
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


### Dimensionality Reduction with Principal Component Analysis (PCA)

Our dataset currently has 100's of columns. To both speed up our training, and improve model accuracy, we can reduce the number of these inputs with dimensionality reduction. Principal Component Analysis converts our original variables to a new set of variables, which are a linear combination of the original set of variables. These variables would be unrelated to each other and hence, are called principal components. Through the use of PCA, we will be using only those variables that capture the maximum variance in the data.

In [11]:
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as H2OPCA

pca_decomp = H2OPCA(k=25, transform="NONE", pca_method="gram_s_v_d", use_all_factor_levels=True)
pca_decomp.train(x=x, training_frame=train)
trainReduced = pca_decomp.predict(train)
trainReduced = trainReduced.cbind(train['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=valid)
validReduced = pca_decomp.predict(valid)
validReduced = validReduced.cbind(valid['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=test)
testReduced = pca_decomp.predict(test)
testReduced = testReduced.cbind(test['Close Tomorrow Trend'])

pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%


### Try AutoML
See what kind of accuracy we can get from automatic model selection and stacking

![](https://image.ibb.co/eJUCPS/modelstacking.png)

In [12]:
from h2o.automl import H2OAutoML

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

# Run AutoML until 20 models are built
autoModel = H2OAutoML(max_models = 10)
autoModel.train(x = x, y = y,
          training_frame = trainReduced,
          validation_frame = validReduced,
          leaderboard_frame = testReduced)

leaders = autoModel.leaderboard
leaders

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,auc,logloss
GBM_grid_0_AutoML_20180501_133925_model_1,0.516477,0.732582
XRT_0_AutoML_20180501_133925,0.513068,0.741701
DeepLearning_0_AutoML_20180501_133925,0.510227,0.749609
StackedEnsemble_BestOfFamily_0_AutoML_20180501_133925,0.5,0.697993
StackedEnsemble_AllModels_0_AutoML_20180501_133925,0.5,0.697993
GBM_grid_0_AutoML_20180501_133925_model_2,0.474432,0.754533
GBM_grid_0_AutoML_20180501_133925_model_3,0.465909,0.751948
DRF_0_AutoML_20180501_133925,0.461364,0.801305
GBM_grid_0_AutoML_20180501_133925_model_4,0.443182,0.721424
GBM_grid_0_AutoML_20180501_133925_model_5,0.440909,1.7167


### Train a Deep Neural Net
Grid search for a good model

![](https://image.ibb.co/esXoyn/neural_Net.png)

In [13]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

hidden_opt = [[75],
              #[100],
              #[200],
              #[200,200],
              #[25,25,25],
              #[50,75,75,50],
              #[75,100,100,75],
              #[1000],
              [500],
              [800]
              ]
l1_opt = [0,0.001]
l2_opt = [0,0.001]
rate_opt= [0.005]
activation_opt=[#"Rectifier",
                "RectifierWithDropout",
                #"Tanh", "TanhWithDropout", 
                #"Maxout", "MaxoutWithDropout"
                ]
epochs_opt=[2500,1500,2000]

hyper_parameters = {"hidden":hidden_opt, 
                    "l1":l1_opt, 
                    "l2":l2_opt,
                    "rate":rate_opt, 
                    "activation":activation_opt,
                    "epochs":epochs_opt}

search_criteria = {"strategy":"RandomDiscrete", "max_models":3, "seed":123456}



model_grid = H2OGridSearch(H2ODeepLearningEstimator(),
                           hyper_params=hyper_parameters,
                           search_criteria=search_criteria,
                           grid_id='test12')
model_grid.train(x=x, 
                 y=y,                                   
                 fast_mode=True,                   
                 stopping_rounds=10, 
                 stopping_tolerance=0,
                 stopping_metric='AUC',
                 adaptive_rate=True,
                 rate_annealing=0.000001,
                 rho=0.95,
                 momentum_ramp=1000000,
                 momentum_stable=0,
                 momentum_start=0,
                 training_frame=trainReduced, 
                 validation_frame=validReduced,
                 score_interval=10)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


### Take a Look at the Best Model

Variable Importances help us see which features in the dataset our model valued most during training. 

In [14]:
model_gridperf1 = model_grid.get_grid(sort_by='recall', decreasing=False)
bestModel = model_gridperf1[0]  


In [15]:
#h2o.save_model(model=bestModel, path="/home/olmsteadbrett/stocks/")
#bestModel = h2o.get_model(leaders[0,0])

In [16]:
#bestModel = h2o.load_model(path="/home/brett/ticker/AutoML/DeepLearning_grid_0_AutoML_20180430_190847_model_15")

In [17]:
importances = bestModel.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

In [18]:
cm = bestModel.confusion_matrix(valid=True)
cm = cm.table.as_data_frame()
confusionMatrix = FF.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

In [19]:
pBestModel = pd.concat([bestModel.predict(test_data=testReduced).as_data_frame(use_pandas=True).reset_index(drop=True), testDF.reset_index(drop=False)], axis =1)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [61]:
prices = df.loc[df['Date'] > split_test]
prices = prices.loc[:,['Date','Close','Close Tomorrow','Close Tomorrow Trend']]
profit = pBestModel.loc[:,['predict']]
profit = pd.concat([prices.reset_index(drop=True), profit.reset_index(drop=True)], axis =1)
profit['predict'] = profit['predict'].astype(np.float64)

profit['profit'] = invest

profit['Best Possible'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['Close Tomorrow Trend'] > 0 else 0, axis=1)

profit['Randomly Invest'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if random.randint(1,1000) > random.randint(1,1000) else 0, axis=1)
profit['profit'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['predict'] > 0 else 0, axis=1)

profit = profit.round(2)

#Print a summary
print "Times " + stockSymbol + " actually went up    : " + str(profit[profit['Close Tomorrow Trend']>0].count()['Close Tomorrow Trend'])
print "Times " + stockSymbol + " predicted to go up  : " + str(profit[profit['predict']>0].count()['predict'])
print "Profit                        : $" + str(profit.profit.sum())
print "Random investment             : $" + str(round(profit['Randomly Invest'].sum(),2))
print "Best possible outcome         : $" + str(round(profit['Best Possible'].sum(),2))

#Plot the profit by day
trace1 = go.Bar(
    x=profit['Date'],
    y=profit['profit'],
    marker = dict(
          color = 'rgb(158,202,225)'
        ),    
    name='Profit'
    
)

data = [trace1]
layout = go.Layout(
    barmode='group',
    title='Profit by Day'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Times PINC actually went up    : 44
Times PINC predicted to go up  : 71
Profit                        : $141.71
Random investment             : $10.52
Best possible outcome         : $564.42


In [21]:
bestModel


Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  test12_model_0


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.254185396033
RMSE: 0.504168023612
LogLoss: 0.730530150537
Mean Per-Class Error: 0.393742808163
AUC: 0.619865908931
Gini: 0.239731817862
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0765877146138: 


,-1,1,Error,Rate
-1,22.0,417.0,0.9499,(417.0/439.0)
1,7.0,381.0,0.018,(7.0/388.0)
Total,29.0,798.0,0.5127,(424.0/827.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0765877,0.6424958,381.0
max f2,0.0173550,0.8161548,397.0
max f0point5,0.5863939,0.5802292,132.0
max accuracy,0.6411729,0.6191052,108.0
max precision,0.8528875,0.8181818,19.0
max recall,0.0173550,1.0,397.0
max specificity,0.9998838,0.9977221,0.0
max absolute_mcc,0.6411729,0.2426611,108.0
max min_per_class_accuracy,0.4360663,0.5876993,205.0
max mean_per_class_accuracy,0.5863939,0.6062572,132.0


Gains/Lift Table: Avg response rate: 46.92 %



...


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-05-01 13:39:47,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-05-01 13:39:47,0.236 sec,44702 obs/sec,10.0,1,8270.0,0.5041680,0.7305302,0.6198659,1.4209622,0.5126965,0.5278669,0.7772403,0.5597884,1.3809524,0.4750958
,2018-05-01 13:39:50,3.923 sec,322375 obs/sec,1500.0,150,1240500.0,0.2110382,0.1849252,0.9956174,2.1314433,0.0193470,0.5971594,1.3739136,0.5223986,1.3809524,0.5134100
,2018-05-01 13:39:50,3.951 sec,322207 obs/sec,1500.0,150,1240500.0,0.5041680,0.7305302,0.6198659,1.4209622,0.5126965,0.5278669,0.7772403,0.5597884,1.3809524,0.4750958


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PC24,1.0,1.0,0.0433249
PC19,0.9995208,0.9995208,0.0433042
PC3,0.9986451,0.9986451,0.0432662
PC10,0.9972117,0.9972117,0.0432041
PC15,0.9904850,0.9904850,0.0429127
---,---,---,---
PC1,0.8312075,0.8312075,0.0360120
PC18,0.8058346,0.8058346,0.0349127
PC17,0.7914104,0.7914104,0.0342878
PC8,0.7747010,0.7747010,0.0335639



See the whole table with table.as_data_frame()
